<a href="https://colab.research.google.com/github/claudevandort/UAI-MachineLearning/blob/master/02_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Aprendizaje Automático

#### Profesor: Gonzalo Ruz
#### Ayudante: Anthony Cho

**Tema**: Clasificador naive Bayes

####Abrir archivo que contiene el dataset CallCenterData

In [1]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/claudevandort/UAI-MachineLearning/master/CallCenterData.csv')

####Resumen con estadística descriptiva

In [2]:
data.describe(include='all')

,ID,Edad,Sexo,NivelEducacion,EstadoCivil,NivelSocEco,HorasLogeadas,HorasHabladas,ContactosEfectivosPromedio,RegistrosTerminados,ProdMayorUbral
count,660.000000,660.000000,660,660,660,660,660.000000,660.000000,660.000000,660.000000,660
unique,NaN,NaN,2,4,2,5,NaN,NaN,NaN,NaN,2
top,NaN,NaN,f,em,s,c3c4,NaN,NaN,NaN,NaN,no
freq,NaN,NaN,340,345,489,386,NaN,NaN,NaN,NaN,351
mean,330.500000,32.034848,NaN,NaN,NaN,NaN,117.025260,41.113746,192.477960,467.773235,NaN
std,190.669872,8.970209,NaN,NaN,NaN,NaN,53.348341,25.310637,146.711340,312.863798,NaN
min,1.000000,19.000000,NaN,NaN,NaN,NaN,0.170000,0.013333,0.000000,0.000000,NaN
25%,165.750000,26.000000,NaN,NaN,NaN,NaN,79.197125,21.767312,84.625000,231.107143,NaN
50%,330.500000,29.500000,NaN,NaN,NaN,NaN,122.679417,37.665000,181.000000,440.500000,NaN
75%,495.250000,36.000000,NaN,NaN,NaN,NaN,160.650250,57.709000,270.558333,645.812500,NaN


####Revisemos la forma de tabla

In [3]:
data.shape

(660, 11)

####Convertir atributos categóricos a numércios

In [4]:
import numpy as np
data["Sexo"]=np.where(data["Sexo"]=="m",0,1)

data["NivelEducacion"]=np.where(data["NivelEducacion"]=="sec",0,
                                np.where(data["NivelEducacion"]=="em",1,
                                         np.where(data["NivelEducacion"]=="ts",2,3)
                                          )
                                 )

data["EstadoCivil"]=np.where(data["EstadoCivil"]=="s",0,1)

data["NivelSocEco"]=np.where(data["NivelSocEco"]=="fue",0,
                                np.where(data["NivelSocEco"]=="e",1,
                                         np.where(data["NivelSocEco"]=="c3c4",2,
                                                  np.where(data["NivelSocEco"]=="c2",3,4)
                                                  
                                                  )
                                          )
                                 )

####Generar la partición entrenamiento-prueba (70% - 30%)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data.iloc[:,0:10],data['ProdMayorUbral'],
    test_size=0.3, random_state=0)
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))

X_train shape: (462, 10)
y_train shape: (462,)
X_test shape: (198, 10)
y_test shape: (198,)


####Entrenar un clasificador NB con  dataset CallCenterData usando sólo variables socio-demograficas y luego evaluar en el conjunto de prueba para generar la matriz de confusion y calcular el accuracy (tasa de clasificaciones correctas)

In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

#selecionar solo variables socio-demográficas
Xtrain=X_train[['Edad','Sexo','NivelEducacion','EstadoCivil','NivelSocEco']]
Xtest=X_test[['Edad','Sexo','NivelEducacion','EstadoCivil','NivelSocEco']]

#codificar la salida
label_encoder = LabelEncoder()
aux=y_train[0]
y_train=label_encoder.fit_transform(y_train)
y_test=label_encoder.fit_transform(y_test)
print("Etiqueta {} codificada como {}".format(aux,y_train[0]))

#entrenar el modelo
model1 = GaussianNB()
model1.fit(Xtrain, y_train)

#realizar la predicción
predicted = model1.predict(Xtest)

#matriz de confusión
print(confusion_matrix(y_test, predicted))

#Accuracy
print("Score: {:.2f}".format(model1.score(Xtest, y_test) ))

Etiqueta si codificada como 1
[[76 27]
 [61 34]]
Score: 0.56


####Veamos como nos va si usamos sólo variables operacionales

In [7]:
#selecionar solo variables operacionales
Xtrain=X_train[['HorasLogeadas','HorasHabladas','ContactosEfectivosPromedio','RegistrosTerminados']]
Xtest=X_test[['HorasLogeadas','HorasHabladas','ContactosEfectivosPromedio','RegistrosTerminados']]


#entrenar el modelo
model2 = GaussianNB()
model2.fit(Xtrain, y_train)

#realizar la predicción
predicted = model2.predict(Xtest)

#matriz de confusión
print(confusion_matrix(y_test, predicted))

#Accuracy
print("Score: {:.2f}".format(model2.score(Xtest, y_test) ))

[[89 14]
 [15 80]]
Score: 0.85


####Y qué pasa si usamos todas las variables?

In [8]:
#Usamos todas las variables menos el ID
Xtrain=X_train.drop(columns ='ID',axis=1)
Xtest=X_test.drop(columns = 'ID',axis=1)


#entrenar el modelo
model3 = GaussianNB()
model3.fit(Xtrain, y_train)

#realizar la predicción
predicted = model3.predict(Xtest)

#matriz de confusión
print(confusion_matrix(y_test, predicted))

#Accuracy
print("Score: {:.2f}".format(model3.score(Xtest, y_test) ))

[[87 16]
 [15 80]]
Score: 0.84


#Qué pasa si uno tiene una variable categórica **nominal**?
* En ese caso se requiere hacer la codificación one-hot (binaria)
* Veamos a modo de ejemplo con la variable EstadoCivil

In [9]:
from sklearn.preprocessing import LabelBinarizer

#carguemos los datos originales nuevamente
data = pd.read_csv('https://raw.githubusercontent.com/claudevandort/UAI-MachineLearning/master/CallCenterData.csv')

lb_style = LabelBinarizer()
lb_results = lb_style.fit_transform(data["NivelEducacion"])                               
NuevasVariables=pd.DataFrame(lb_results, columns=lb_style.classes_)
NuevasVariables.head(10)

,em,sec,ts,un
0,0,0,0,1
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,0,0,0,1
5,1,0,0,0
6,1,0,0,0
7,1,0,0,0
8,0,0,1,0
9,1,0,0,0


####Luego concatenamos ambas tablas

In [10]:
data2=pd.concat([data,NuevasVariables], axis=1)
data2.head(10)

,ID,Edad,Sexo,NivelEducacion,EstadoCivil,NivelSocEco,HorasLogeadas,HorasHabladas,ContactosEfectivosPromedio,RegistrosTerminados,ProdMayorUbral,em,sec,ts,un
0,1,32,m,un,s,c3c4,151.152500,45.852500,192.250000,591.750000,si,0,0,0,1
1,2,24,m,em,s,c3c4,71.180000,24.760000,1.000000,501.000000,no,1,0,0,0
2,3,29,m,em,c,c3c4,120.560000,49.450000,156.000000,993.000000,si,1,0,0,0
3,4,48,f,em,c,e,90.850000,28.850000,442.000000,569.000000,no,1,0,0,0
4,5,25,m,un,s,c3c4,190.180000,41.160000,0.000000,0.000000,si,0,0,0,1
5,6,30,m,em,s,c3c4,164.458333,32.273333,186.666667,359.333333,si,1,0,0,0
6,7,43,f,em,c,e,120.038750,38.265000,105.500000,209.375000,si,1,0,0,0
7,8,31,m,em,c,c3c4,67.733333,23.303333,49.333333,63.000000,si,1,0,0,0
8,9,29,f,ts,s,c3c4,68.986667,27.370000,155.333333,418.333333,no,0,0,1,0
9,10,47,m,em,s,e,168.875000,80.841667,452.000000,915.833333,si,1,0,0,0
